In [ ]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
from train import *


In [2]:
annotation_path = '2007_trainval.txt'
log_dir = 'logs/000/'
weights_path='model_data/yolo_weights.h5'
classes_path = 'model_data/voc_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)
val_split = 0.1


input_shape = (608, 608) # multiple of 32, hw

is_tiny_version = len(anchors)==6 # default setting

In [3]:
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

model = create_model(input_shape, anchors, num_classes,
                     freeze_body=2, weights_path=weights_path) # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

Create YOLOv3 model with 9 anchors and 2 classes.


/anaconda3/envs/py36-test/lib/python3.6/site-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 21) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/anaconda3/envs/py36-test/lib/python3.6/site-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/anaconda3/envs/py36-test/lib/python3.6/site-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/anaconda3/envs/py36-test/lib/python3.6/site-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/anaconda3/envs/py36-test/lib/python3.6/site-packages/keras/engine/topology.py:3462: UserW

Load weights model_data/yolo_weights.h5.
Freeze the first 249 layers of total 252 layers.


In [ ]:
# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a not bad model.

model.compile(optimizer=Adam(lr=1e-3), loss={
    # use custom yolo_loss Lambda layer.
    'yolo_loss': lambda y_true, y_pred: y_pred})

batch_size = 32
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=50,
        initial_epoch=0,
        callbacks=[logging, checkpoint])
model.save_weights(log_dir + 'trained_weights_stage_1.h5')

Train on 68 samples, val on 7 samples, with batch size 32.
Epoch 1/50
2/2 [==============================] - 513s 257s/step - loss: 18066.5576 - val_loss: 15950.8135
Epoch 2/50
2/2 [==============================] - 445s 222s/step - loss: 14072.2241 - val_loss: 13027.6123
Epoch 3/50
2/2 [==============================] - 519s 259s/step - loss: 11045.7539 - val_loss: 9936.9570
Epoch 4/50
2/2 [==============================] - 451s 226s/step - loss: 8624.7292 - val_loss: 8144.3804
Epoch 5/50
2/2 [==============================] - 2292s 1146s/step - loss: 6861.9880 - val_loss: 6431.7705
Epoch 6/50
2/2 [==============================] - 595s 298s/step - loss: 5331.5237 - val_loss: 4845.4258
Epoch 7/50
2/2 [==============================] - 940s 470s/step - loss: 4259.4784 - val_loss: 3784.4165
Epoch 8/50
2/2 [==============================] - 509s 255s/step - loss: 3639.3052 - val_loss: 3137.2805
Epoch 9/50
2/2 [==============================] - 467s 234s/step - loss: 2815.9014 - val_loss:

In [ ]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')

batch_size = 32 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=100,
    initial_epoch=50,
    callbacks=[logging, checkpoint, reduce_lr, early_stopping])
model.save_weights(log_dir + 'trained_weights_final.h5')

# Further training if needed.